# Trade Misinvoicing: Bilateral Trade Gaps

## _A quick and easy method for calculating bilateral trade disparities using Comtrade data and Python_

### Downloading Data

1. Go to the UN Comtrade data portal (https://comtrade.un.org/data/)
2. Add both countries in your bilateral trade pair to the "Reporter" and "Partner" fields
3. Select "All" in the "trade flows" fields
4. Select year or years of interest
5. Select "All 4-Digit HS Commodity Codes" in the "HS commodity codes" field
6. Dowload the data as a csv


In this example, we will work with data from China and Myanmar from 2018 to 2020.

In [1]:
#load libraries

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', None)

In [5]:
#create dataframe using Comtrade csv file

df = pd.read_csv('china-myanmar_2017-2020.csv')

One way to quickly take stock of the data is the .info( ) command. This gives us column names, number of values per column, and the data type

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9750 entries, 0 to 9749
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Classification          9750 non-null   object 
 1   Year                    9750 non-null   int64  
 2   Period                  9750 non-null   int64  
 3   Period Desc.            9750 non-null   int64  
 4   Aggregate Level         9750 non-null   int64  
 5   Is Leaf Code            9750 non-null   int64  
 6   Trade Flow Code         9750 non-null   int64  
 7   Trade Flow              9750 non-null   object 
 8   Reporter Code           9750 non-null   int64  
 9   Reporter                9750 non-null   object 
 10  Reporter ISO            9750 non-null   object 
 11  Partner Code            9750 non-null   int64  
 12  Partner                 9750 non-null   object 
 13  Partner ISO             9750 non-null   object 
 14  2nd Partner Code        0 non-null      

### Preparing Data

We will begin by analyzing disparities in exports reported by Myanmar to the UN and imports reported by China to the UN in 2020. 

First, we will need to break our master dataframe into subsets: one for Myanmar's exports and one for China's imports in 2020. For the subsets, we will include commodity code and description, net weight, and dollar value.  

Then, we will merge the two dataframes into one and generate a new column for our trade disparity analysis.

In [10]:
#DATA PREP

#Create dataframe subsets for each country and trade flow direction

MMR_EX = df[(df['Trade Flow'] == 'Export') & (df['Reporter ISO'] == 'MMR') & (df.Year == 2020)]\
[['Commodity Code', 'Commodity', 'Netweight (kg)', 'Trade Value (US$)']]

CHN_IM = df[(df['Trade Flow'] == 'Import') & (df['Reporter ISO'] == 'CHN') & (df.Year == 2020)]\
[['Commodity Code', 'Netweight (kg)', 'Trade Value (US$)']]


#Rename Columns

MMR_EX.rename(columns = {'Trade Value (US$)' : 'MMR_EX$', 'Netweight (kg)' : 'MMR_EXkg'}, inplace = True)
CHN_IM.rename(columns = {'Trade Value (US$)' : 'CHN_IM$', 'Netweight (kg)' : 'CHN_IMkg'}, inplace = True)

#merge datasets
##China Imports-Laos Exports

gap2019 = pd.merge(CHN_IM, MMR_EX, on = 'Commodity Code')


#calculate trade differential (exports*1.1 reflects 10% of FOB value to account for cost of insurance and freight)
##10% is a conservative estimate compared to OECD averages for insurance and freight costs
##(See https://stats.oecd.org/viewhtml.aspx?datasetcode=CIF_FOB_ITIC&lang=en)

gap2019['GAP$'] = gap2019['CHN_IM$'] - (gap2019['MMR_EX$']*1.1)

#rearrange columns so 'commodity code' is next to 'commodity'

gap2019 = gap2019[['Commodity Code', 'Commodity', 'CHN_IM$', 'MMR_EX$', 'CHN_IMkg', 'MMR_EXkg', 'GAP$']].set_index('Commodity Code', inplace=True)

### Export Underinvoicing in Myanmar/Import Overinvoicing in China

In our gap2019 dataframe we have generated a new column,'GAP2019,' which is the difference for each 4-digit HS commodity code between the value of Chinese imports from Myanmar, as reported to the UN, and the value of Myanmar exports to China, as reported to the UN.

Some of these values are negative. This would suggest that China under-reported the value of its imports from Myanmar, or that Myanmar over-reported the value of its exports to China. Others of these values are positive, which would reflect the opposite. 

As this range of values indicates, trade misinvoicing can occur via four mechanisms: 

1. Export over-invoicing
2. Import under-invoicing
3. Export under-invoicing
4. Import over-invoicing

Capital flight typically occurs via export under-invoicing and import over-invoicing. These two methods of trade misinvoicing would correspond to positive values in our GAP2019 column.



In [20]:
#This table ranks all positive 4-digit commodity codes in descending order by the size of trade disparities between China Imports and Laos Exports

gap2019.loc[gap2019['GAP$']>0].sort_values(by='GAP$', ascending=False)

,Commodity,CHN_IM$,MMR_EX$,CHN_IMkg,MMR_EXkg,GAP$
Commodity Code,,,,,,
9999,Commodities not specified according to kind,1289303945,819219,0.000000e+00,0.0,1.288403e+09
2609,Tin ores and concentrates,446813944,6538106,1.450770e+08,737478.0,4.396220e+08
2601,Iron ores and concentrates; including roasted iron pyrites,90471730,573323,2.770845e+09,7611220.0,8.984107e+07
4001,"Natural rubber, balata, gutta-percha, guayule, chicle and similar gums; in primary forms or in plates, sheets or strip",209170240,129432523,1.455495e+08,61029959.0,6.679446e+07
1212,"Locust beans, seaweeds and other algae, sugar beet, sugar cane, fresh, chilled, frozen or dried, whether or not ground; fruit stones, kernels and other vegetable products (including unroasted chicory roots) used primarily for human consumption, n.e.c.",53445394,8601341,9.125420e+08,71656300.0,4.398392e+07
2608,Zinc ores and concentrates,40723130,251160,7.069210e+07,3137660.0,4.044685e+07
2607,Lead ores and concentrates,74005836,32109909,1.091610e+08,21687800.0,3.868494e+07
2301,"Flours, meal and pellets, of meat or meat offal, of fish or of crustaceans, molluscs or other aquatic invertebrates, unfit for human consumption; greaves",32140550,210490,2.516300e+07,194000.0,3.190901e+07
7402,"Copper; unrefined, copper anodes for electrolytic refining",20574193,87500,3.814560e+06,25000.0,2.047794e+07


In [ ]:
#export table to csv (remove hash below to run command)

#gap2019.loc[gap2019['GAP$']>0].sort_values(by='GAP$', ascending=False).to_csv('MMR-CHN_MMR-export-underinvoicing.csv')